In [108]:
from random import randint, sample
from hypergraphs import *
from qf import graphs
import networkx as nx


n = 7            # Number of nodes of BH
m = 5            # Number of hyperarcs of BH
multiplicity = 6 # Maximum size of a fibre over BH

BH = hnx.Hypergraph()   # Generate BH

for i in range(m):
    k = randint(1, n)  # Number of sources
    sources = sample(range(0, n), k)
    target = randint(0, n)
    add_directed_hyperedge(BH, sources, target, f"h_{i}")

In [109]:
B, dd = hg2g(BH) # Lift every node
lifting = {}
for x in B.nodes():
    if dd[x] == 0: # Node, liftable
        lifting[x]=[i for i in range(randint(1, multiplicity))]   # Multiplicity of nodes

H = hnx.Hypergraph() # Generate H appropriately
for h in BH.edges():
    name = str(h)
    target = h.props['target']
    sources = set([x for x in h.elements])-set([target])
    for t in lifting[target]:
        ss = []
        for s in sources:
            s_lift = sample(lifting[s], 1)[0]
            ss += [(s, s_lift)]
        add_directed_hyperedge(H, ss, (target, t), "{}_{}".format(name, t))


In [110]:
rename = {}
s = {}
for h in H.edges():
    tn = h.props['target']
    t = f"x_{tn[0]}_{tn[1]}"
    if t not in rename:
        rename[t] = f"y_{len(rename)}"
    tr = rename[t]
    sources = set([f"x_{x[0]}_{x[1]}" for x in h.elements])
    zources = []
    for z in sources:
        if not z in rename:
            rename[z] = f"y_{len(rename)}"
        zources += [rename[z]]
    if tr in s:
        s[tr] += [zources]
    else:
        s[tr] = [zources]
for k, v in s.items():
    print(f"{k}' = ", " + ".join([" * ".join([vv for vv in list(x)]) for x in v]))

print()
print(rename)

y_0' =  y_0 * y_1 + y_0 * y_4 * y_5 * y_6 * y_7 * y_1
y_2' =  y_2 * y_1 + y_8 * y_5 * y_6 * y_2 * y_9 * y_1
y_3' =  y_3 * y_1 + y_10 * y_5 * y_11 * y_12 * y_1 * y_3
y_13' =  y_5 * y_14 * y_9 * y_12 * y_1 * y_13 * y_3
y_11' =  y_10 * y_5 * y_15 * y_11 * y_12 * y_1 * y_3
y_16' =  y_0 * y_5 * y_14 * y_16 * y_7 * y_9 * y_1
y_17' =  y_8 * y_5 * y_18 * y_2 * y_17 * y_19 * y_1
y_20' =  y_5 * y_21 * y_2 * y_15 * y_20 * y_12 * y_1
y_6' =  y_4 * y_5 * y_6 * y_15 * y_12 * y_1 * y_3
y_5' =  y_9 * y_5 * y_12 * y_1
y_22' =  y_18 * y_22 * y_5 * y_3
y_23' =  y_18 * y_0 * y_5 * y_23

{'x_5_0': 'y_0', 'x_2_0': 'y_1', 'x_5_1': 'y_2', 'x_5_2': 'y_3', 'x_0_5': 'y_4', 'x_1_0': 'y_5', 'x_6_5': 'y_6', 'x_3_2': 'y_7', 'x_3_1': 'y_8', 'x_0_3': 'y_9', 'x_0_4': 'y_10', 'x_6_1': 'y_11', 'x_3_0': 'y_12', 'x_6_0': 'y_13', 'x_4_0': 'y_14', 'x_4_2': 'y_15', 'x_6_2': 'y_16', 'x_6_3': 'y_17', 'x_4_1': 'y_18', 'x_0_2': 'y_19', 'x_6_4': 'y_20', 'x_0_1': 'y_21', 'x_7_0': 'y_22', 'x_7_1': 'y_23'}


In [111]:
s = {}
for h in BH.edges():
    tn = h.props['target']
    tr = f"x_{tn}"
    sources = set([f"x_{x}" for x in h.elements])
    if tr in s:
        s[tr] += [sources]
    else:
        s[tr] = [sources]
for k, v in s.items():
    print(f"{k}' = ", " + ".join([" * ".join([vv for vv in list(x)]) for x in v]))


x_5' =  x_2 * x_5 + x_3 * x_5 * x_0 * x_1 * x_2 * x_6
x_6' =  x_3 * x_5 * x_0 * x_1 * x_2 * x_4 * x_6
x_1' =  x_3 * x_2 * x_0 * x_1
x_7' =  x_5 * x_4 * x_1 * x_7
